In [1]:
from nltk.stem.porter import PorterStemmer
stemer=PorterStemmer()
' '.join([stemer.stem(w).strip("'") for w in "dish washer's washed dishes".split()])

'dish washer wash dish'

In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sa=SentimentIntensityAnalyzer()
sa.lexicon

{'$:': -1.5,
 '%)': -0.4,
 '%-)': -1.5,
 '&-:': -0.4,
 '&:': -0.7,
 "( '}{' )": 1.6,
 '(%': -0.9,
 "('-:": 2.2,
 "(':": 2.3,
 '((-:': 2.1,
 '(*': 1.1,
 '(-%': -0.7,
 '(-*': 1.3,
 '(-:': 1.6,
 '(-:0': 2.8,
 '(-:<': -0.4,
 '(-:o': 1.5,
 '(-:O': 1.5,
 '(-:{': -0.1,
 '(-:|>*': 1.9,
 '(-;': 1.3,
 '(-;|': 2.1,
 '(8': 2.6,
 '(:': 2.2,
 '(:0': 2.4,
 '(:<': -0.2,
 '(:o': 2.5,
 '(:O': 2.5,
 '(;': 1.1,
 '(;<': 0.3,
 '(=': 2.2,
 '(?:': 2.1,
 '(^:': 1.5,
 '(^;': 1.5,
 '(^;0': 2.0,
 '(^;o': 1.9,
 '(o:': 1.6,
 ")':": -2.0,
 ")-':": -2.1,
 ')-:': -2.1,
 ')-:<': -2.2,
 ')-:{': -2.1,
 '):': -1.8,
 '):<': -1.9,
 '):{': -2.3,
 ');<': -2.6,
 '*)': 0.6,
 '*-)': 0.3,
 '*-:': 2.1,
 '*-;': 2.4,
 '*:': 1.9,
 '*<|:-)': 1.6,
 '*\\0/*': 2.3,
 '*^:': 1.6,
 ',-:': 1.2,
 "---'-;-{@": 2.3,
 '--<--<@': 2.2,
 '.-:': -1.2,
 '..###-:': -1.7,
 '..###:': -1.9,
 '/-:': -1.3,
 '/:': -1.3,
 '/:<': -1.4,
 '/=': -0.9,
 '/^:': -1.0,
 '/o:': -1.4,
 '0-8': 0.1,
 '0-|': -1.2,
 '0:)': 1.9,
 '0:-)': 1.4,
 '0:-3': 1.5,
 '0:03': 1.9,
 '

In [3]:
[(tok,score) for tok,score in sa.lexicon.items() if " " in tok]

[("( '}{' )", 1.6),
 ("can't stand", -2.0),
 ('fed up', -1.8),
 ('screwed up', -1.5)]

In [4]:
sa.polarity_scores(text="Python is very readable and it's great for NLP.")

{'neg': 0.0, 'neu': 0.661, 'pos': 0.339, 'compound': 0.6249}

In [5]:
sa.polarity_scores(text="Python is not a bad choice for most application.")

{'neg': 0.0, 'neu': 0.737, 'pos': 0.263, 'compound': 0.431}

In [6]:
corpus=["Absolutely perfect! Love it! :-] :-]","Horrible Completely useless. :(","It was OK. Some good and some bad things."]
for doc in corpus:
    scores=sa.polarity_scores(doc)
    print("{:+}:{}".format(scores['compound'],doc))

+0.8766:Absolutely perfect! Love it! :-] :-]
-0.868:Horrible Completely useless. :(
-0.1531:It was OK. Some good and some bad things.


In [7]:
from nlpia.data.loaders import get_data
movies=get_data("hutto_movies")
movies.head().round(2)

,sentiment,text
id,,
1,2.27,The Rock is destined to be the 21st Century's ...
2,3.53,The gorgeously elaborate continuation of ''The...
3,-0.60,Effective but too tepid biopic
4,1.47,If you sometimes like to go to the movies to h...
5,1.73,"Emerges as something rare, an issue movie that..."


In [8]:
movies.describe().round(2)

,sentiment
count,10605.00
mean,0.00
std,1.92
min,-3.88
25%,-1.77
50%,-0.08
75%,1.83
max,3.94


In [9]:
import pandas as pd
pd.set_option('display.width',75)
from nltk.tokenize import casual_tokenize
bag_of_words=[]
from collections import Counter
for text in movies.text:
    bag_of_words.append(Counter(casual_tokenize(text)))
df_bows=pd.DataFrame.from_dict(bag_of_words)
df_bows=df_bows.fillna(0).astype(int)
df_bows.shape


(10605, 20756)

In [10]:
df_bows.head()

,The,Rock,is,destined,to,be,the,21st,Century's,new,...,Ill,slummer,Rashomon,dipsticks,Bearable,Staggeringly,’,ve,muttering,dissing
0,1,1,1,1,2,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,4,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()
nb=nb.fit(df_bows,movies.sentiment<0)

In [12]:
movies['predicted_sentiment']=0
movies['error']=0
movies['predicted_sentiment']=nb.predict_proba(df_bows)*8-4
movies['error']=(movies.predicted_sentiment-movies.sentiment).abs()

In [13]:
ss=movies.iloc[[5],[1]].values[0][0]
ss

'The film provides some great insight into the neurotic mindset of all comics    even those who have reached the absolute top of the game.'

In [14]:
movies["sentiment_ispositive"]=(movies.sentiment>0).astype(int)


In [15]:
movies["predicted_ispositive"]=(movies.predicted_sentiment>0).astype(int)


In [16]:
movies['''sentiment predicted_sentiment sentiment_ispositive predicted_ispositive'''.split()].head(8)

,sentiment,predicted_sentiment,sentiment_ispositive,predicted_ispositive
id,,,,
1,2.266667,2.291318,1,1
2,3.533333,3.999875,1,1
3,-0.600000,-3.656504,0,0
4,1.466667,2.025277,1,1
5,1.733333,3.911166,1,1
6,2.533333,3.995434,1,1
7,2.466667,3.958007,1,1
8,1.266667,-1.999168,1,0


In [17]:
movies.head(8)

,sentiment,text,predicted_sentiment,error,sentiment_ispositive,predicted_ispositive
id,,,,,,
1,2.266667,The Rock is destined to be the 21st Century's ...,2.291318,0.024651,1,1
2,3.533333,The gorgeously elaborate continuation of ''The...,3.999875,0.466542,1,1
3,-0.600000,Effective but too tepid biopic,-3.656504,3.056504,0,0
4,1.466667,If you sometimes like to go to the movies to h...,2.025277,0.558611,1,1
5,1.733333,"Emerges as something rare, an issue movie that...",3.911166,2.177832,1,1
6,2.533333,The film provides some great insight into the ...,3.995434,1.462101,1,1
7,2.466667,Offers that rare combination of entertainment ...,3.958007,1.491340,1,1
8,1.266667,Perhaps no picture ever made has more literall...,-1.999168,3.265835,1,0


In [18]:
movies.error.mean()

1.870573773176738

In [19]:
(movies.predicted_ispositive==movies.sentiment_ispositive).sum()/len(movies)

0.9322017916077322

In [21]:
df_bows.columns.sort()

TypeError: cannot sort an Index object in-place, use sort_values instead

In [23]:
df_bows

,The,Rock,is,destined,to,be,the,21st,Century's,new,...,Ill,slummer,Rashomon,dipsticks,Bearable,Staggeringly,’,ve,muttering,dissing
0,1,1,1,1,2,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,4,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10600,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10601,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10602,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10603,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,2,1,0,0


In [24]:
products=get_data('hutto_products')
bag_of_words1=[]
for text in products.text:
    bag_of_words1.append(casual_tokenize(text))
df_procducts_bows=pd.DataFrame.from_dict(bag_of_words1)
df_procducts_bows

,0,1,2,3,4,5,6,7,8,9,...,1338,1339,1340,1341,1342,1343,1344,1345,1346,1347
0,troubleshooting,ad,-,2500,and,ad,-,2600,no,picture,...,None,None,None,None,None,None,None,None,None,None
1,repost,from,january,13,",",2004,with,a,better,fit,...,None,None,None,None,None,None,None,None,None,None
2,does,your,apex,dvd,player,only,play,dvd,audio,without,...,None,None,None,None,None,None,None,None,None,None
3,or,does,it,play,audio,and,video,but,scrolling,in,...,None,None,None,None,None,None,None,None,None,None
4,before,you,try,to,return,the,player,or,waste,hours,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3541,the,other,day,when,i,was,listening,to,a,song,...,None,None,None,None,None,None,None,None,None,None
3542,it,says,i,have,a,harddisk,problem,.,None,None,...,None,None,None,None,None,None,None,None,None,None
3543,and,since,i'm,out,here,i,can't,mail,it,back,...,None,None,None,None,None,None,None,None,None,None
3544,it,worked,good,for,a,while,.,None,None,None,...,None,None,None,None,None,None,None,None,None,None
